# Evaluación de simplificación de textos (Prompting + LLM)

Fichero debe contener las siguientes columnas (en el notebook se pueden renombrar):
- COL_ID = "text_id"

- COL_COMPLEX = "original"
- COL_REFERENCE = "reference"
- COL_PREDICTION = "prediction"



**Output:** guarda un CSV con los scores:
`scores_{dataset}_{model}_{tipo_prompt}_{strategy_prompt}.csv`



### 📊 Métricas de simplificación:

- BLEU,
- SARI,
- BERTScore,
- MeaningBERT,


### 📊 Métricas de Legibilidad

- FKGL: Nivel escolar para comprender el texto. Ej 8.2 ≈ 8º grado
- Flesch Reading Ease: Facilidad de lectura basada en longitud de frases y sílablas. Más alto = más fácil.


| Métrica | Adaptación en español | Adaptación en francés |
|----------|----------------------|----------------------|
| FKGL (Flesch–Kincaid Grade Level) | No | No |
| Flesch Reading Ease | Fernández-Huerta | Kandel & Moles |






## 1) Configuración


In [6]:
#@title Parámetros (edita si lo necesitas)
PRED_FILE = "predictions_gemma-3-12b-it_docs_0.csv"    #@param {type:"string"}

DATASET = "tsar2025"          #@param ["tsar2025","plaba","cochrane_en", "clara-med","cochrane_fr","meds","wiki"]

MODEL_NAME = "gemma-3-12b-it" #@param {type:"string"}
PROMPT_STRATEGY = "one"      #@param ["zero","one","few"]
PROMPT_TYPE = "minimal"           #@param ["minimal", "ISO", "ISO+examples"]


# Si tu CSV usa otros nombres, edítalos aquí.
COL_PAIR_ID = "text_id"
COL_COMPLEX = "original"
COL_REFERENCE = "reference"
COL_PREDICTION = "prediction"


lang='en'
DATASET2LANG_DEFAULT = {
    "tsar2025": "en",
    "plaba": "en",
    "cochrane_en": "en",

    "clara-med": "es",
    "cochrane_fr": "fr",
    "meds": "fr",
    "wiki": "fr",
}
if DATASET in DATASET2LANG_DEFAULT.keys():
    lang = DATASET2LANG_DEFAULT[DATASET]
print(DATASET,lang)


tsar2025 en


## 2) Instalación de dependencias


In [3]:
!pip -q install sacrebleu bert-score textstat sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.2 MB/s eta 0:00:00


## 0) Carga de fichero y dataset


In [7]:
import pandas as pd
print("✅ file_path =", PRED_FILE)

df = pd.read_csv(PRED_FILE)
df = df[[COL_PAIR_ID, COL_COMPLEX, COL_REFERENCE, COL_PREDICTION]].copy()


print("✅ Dataset cargado. Tamaño:", df.shape)
# print(df.head(1))
# Extraer columnas
sources = df[COL_COMPLEX].astype(str).tolist()
references = df[COL_REFERENCE].astype(str).tolist()
predictions = df[COL_PREDICTION].astype(str).tolist()

✅ file_path = predictions_gemma-3-12b-it_docs_0.csv
✅ Dataset cargado. Tamaño: (117, 4)


## 4) Métricas de simplificación (BLEU, SARI, BERTScore, MeaningBERT, LENS)


### BLEU Y SARI


In [8]:
import sacrebleu
import numpy as np

scores={}

# ======================
# 🔵 BLEU
# ======================
# BLEU
bleu = sacrebleu.corpus_bleu(predictions, [references]).score
print(f"BLEU: {bleu:.2f}")
scores["BLEU"]=round(bleu,2)

# ======================
# 🟢 SARI  (implementación sin easse)
# ======================
def sari_score(orig_sents, sys_sents, refs_sents):
    def get_ngrams(sentence, n):
        tokens = sentence.split()
        return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

    def f1(tp, denom_p, denom_r):
        p = tp / denom_p if denom_p else 0.0
        r = tp / denom_r if denom_r else 0.0
        return (2*p*r/(p+r)) if (p+r) else 0.0

    total = 0.0
    for orig, sys, ref in zip(orig_sents, sys_sents, refs_sents):
        add, keep, dele = [], [], []
        for n in range(1, 5):
            o = set(get_ngrams(orig, n))
            s = set(get_ngrams(sys, n))
            r = set(get_ngrams(ref, n))

            # ADD
            s_add = s - o
            r_add = r - o
            tp = len(s_add & r_add)
            add.append(f1(tp, len(s_add), len(r_add)))

            # KEEP
            s_keep = s & o
            r_keep = r & o
            tp = len(s_keep & r_keep)
            keep.append(f1(tp, len(s_keep), len(r_keep)))

            # DELETE
            s_del = o - s
            r_del = o - r
            tp = len(s_del & r_del)
            dele.append(f1(tp, len(s_del), len(r_del)))

        total += (np.mean(add) + np.mean(keep) + np.mean(dele)) / 3.0

    return (total / len(orig_sents)) * 100.0 if orig_sents else 0.0


sari = sari_score(sources, predictions, references)
print(f"SARI: {sari:.2f}")
scores["SARI"]=round(sari,2)


BLEU: 5.36
SARI: 33.32


### BERT_SCORE

- Para inglés, usamos *microsoft/deberta-xlarge-mnli*, porque es el que mejores correlaciones con la evaluación humana (ver https://pypi.org/project/bert-score/)

- Para el resto, usamos *xlm-roberta-large* porque da mejor correlación que el bert-base.

In [9]:
from bert_score import score

model_type="xlm-roberta-large"

#if lang=='en':
    # es el modelo que mejor correlación tiene con la ev. humana
    # model_type='microsoft/deberta-xlarge-mnli'


print("🤖 Modelo seleccionado:", model_type)

P, R, F1 = score(predictions, references, model_type=model_type,
    lang=lang,  # cambia si no es español
    rescale_with_baseline=True, # normalizes scores for interpretability
    verbose=True
)

precision = round(P.mean().item(),4)
recall = round(R.mean().item(),4)
f1 = round(F1.mean().item(),4)

print("BERTScore Precision:", precision)
print("BERTScore Recall:", recall)
print("BERTScore F1:", f1)
scores["BERTScore-P:"]=precision
scores["BERTScore-R:"]=recall
scores["BERTScore-F1:"]=f1

🤖 Modelo seleccionado: xlm-roberta-large


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.92 seconds, 11.79 sentences/sec
BERTScore Precision: 0.2111
BERTScore Recall: -0.0606
BERTScore F1: 0.0707


### MeaningBERT


In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

complex_emb = embedder.encode(sources, convert_to_numpy=True, batch_size=32)
ref_emb = embedder.encode(references, convert_to_numpy=True, batch_size=32)
pred_emb = embedder.encode(predictions, convert_to_numpy=True, batch_size=32)

sim_OR_diag = cosine_similarity(complex_emb, ref_emb).diagonal()
sim_OR = round(np.mean(sim_OR_diag),4)
print("MeaningBERT (complex → simple):", sim_OR)
scores["MBERT_OR"]=sim_OR

sim_OP_diag = cosine_similarity(complex_emb, pred_emb).diagonal()
sim_OP = round(np.mean(sim_OP_diag),4)
print("MeaningBERT (complex → prediction):", sim_OP)
scores["MBERT_OP"]=sim_OP

sim_RP_diag = cosine_similarity(ref_emb, pred_emb).diagonal()
sim_RP = round(np.mean(sim_RP_diag),4)
print("MeaningBERT (simple → prediction):", sim_RP)
scores["MBERT_RP"]=sim_RP




modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

MeaningBERT (complex → simple): 0.7774
MeaningBERT (complex → prediction): 0.6879
MeaningBERT (simple → prediction): 0.6239


## 6) Métricas de legibilidad (según idioma)


In [13]:
import numpy as np
import textstat

if lang not in {"en","es","fr"}:
    print(f"⚠️ Idioma '{lang}' no soportado en este notebook. Usaré 'en'.")
    lang = "en"

try:
    textstat.set_lang(lang)
except Exception:
    pass

def kandel_moles_french(text: str) -> float:
    """Kandel & Moles (adaptación francesa de Flesch): 207 - 1.015*ASL - 73.6*ASW"""
    sentences = max(1, textstat.sentence_count(text))
    words = max(1, textstat.lexicon_count(text, removepunct=True))
    syllables = max(1, textstat.syllable_count(text))
    asl = words / sentences
    asw = syllables / words
    return 207.0 - (1.015 * asl) - (73.6 * asw)

def avg_metric(texts, fn):
    vals = []
    for t in texts:
        try:
            vals.append(float(fn(t)))
        except Exception:
            vals.append(np.nan)
    return float(np.nanmean(vals))

pred_texts = predictions

if lang == "en":
    scores["FKGL"] = avg_metric(pred_texts, textstat.flesch_kincaid_grade)
    scores["Flesch"] = avg_metric(pred_texts, textstat.flesch_reading_ease)
    scores["SMOG"] = avg_metric(pred_texts, textstat.smog_index)

elif lang == "es":
    scores["FernandezHuerta"] = avg_metric(pred_texts, getattr(textstat, "fernandez_huerta", lambda x: np.nan))
    scores["SzigrisztPazos"]  = avg_metric(pred_texts, getattr(textstat, "szigriszt_pazos", lambda x: np.nan))
    scores["GutierrezPolini"] = avg_metric(pred_texts, getattr(textstat, "gutierrez_polini", lambda x: np.nan))

elif lang == "fr":
    scores["KandelMoles"] = float(np.nanmean([kandel_moles_french(t) for t in pred_texts]))

scores


{'BLEU': 5.36,
 'SARI': np.float64(33.32),
 'BERTScore-P:': 0.2111,
 'BERTScore-R:': -0.0606,
 'BERTScore-F1:': 0.0707,
 'MBERT_OR': np.float32(0.7774),
 'MBERT_OP': np.float32(0.6879),
 'MBERT_RP': np.float32(0.6239),
 'FKGL': 6.521908516060169,
 'Flesch': 67.74894170928633,
 'SMOG': 8.696261850221031}

## 7) Mostrar resultados y guardar a CSV


In [14]:
import pandas as pd
from datetime import datetime

meta = {
    "dataset": DATASET,
    "model": MODEL_NAME,
    "prompt_type": PROMPT_TYPE,
    "prompt_strategy": PROMPT_STRATEGY,
    "lang": lang,
    "n": len(df),
    "timestamp": datetime.now().isoformat(timespec="seconds"),
}

row = {**meta, **scores}
res_df = pd.DataFrame([row])

print("=== Resultados ===")
display(res_df)

out_name = f"scores_{DATASET}_{MODEL_NAME}_{PROMPT_TYPE}_{PROMPT_STRATEGY}.csv"
res_df.to_csv(out_name, index=False)
print("✅ Guardado:", out_name)


=== Resultados ===


,dataset,model,prompt_type,prompt_strategy,lang,n,timestamp,BLEU,SARI,BERTScore-P:,BERTScore-R:,BERTScore-F1:,MBERT_OR,MBERT_OP,MBERT_RP,FKGL,Flesch,SMOG
0,tsar2025,gemma-3-12b-it,minimal,one,en,117,2026-03-01T19:16:22,5.36,33.32,0.2111,-0.0606,0.0707,0.7774,0.6879,0.6239,6.521909,67.748942,8.696262


✅ Guardado: scores_tsar2025_gemma-3-12b-it_minimal_one.csv
